In [2]:
import time

# Global Rankings

Testing the lambda function for the global ranking endpoint.

In [3]:
import boto3
import pandas as pd
import json
import time

def run_query(query, database, s3_output):
    client = boto3.client('athena')
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': s3_output,
        }
    )
    return response['QueryExecutionId']

def get_results(query_id):
    client = boto3.client('athena')
    
    while True:
        response = client.get_query_execution(QueryExecutionId=query_id)
        if response['QueryExecution']['Status']['State'] == 'SUCCEEDED':
            break
        elif response['QueryExecution']['Status']['State'] == 'FAILED':
            raise Exception("Athena query failed!")
        time.sleep(2)
    
    result = client.get_query_results(QueryExecutionId=query_id)
    return result

client = boto3.client('athena')

def lambda_handler(event, context):

    # Athena query
    query = """

WITH unnested_tournaments AS (
    SELECT 
        id AS league_id,
        region,
        tournament.id AS tournament_id
    FROM 
        lol.leagues
        CROSS JOIN UNNEST(tournaments) AS t (tournament)
),
tourney AS (
    SELECT * 
    FROM lol.tournaments
    WHERE startdate > '2001-01-01'
    AND startdate < '2023-12-12'
),
tourney_matches AS (
    SELECT 
        t.*,
        tr.region,
        stage.name AS stage_name,
        stage.type AS stage_type,
        stage.slug AS stage_slug,
        section.name AS section_name,
        match_item.id AS match_id,
        match_item.type AS match_type,
        match_item.state AS match_state,
        match_item.mode AS match_mode,
        match_item.strategy.type AS match_strategy_type,
        match_item.strategy.count AS match_strategy_count,
        team.id AS team_id,
        team.side AS team_side,
        team.record.wins AS team_wins,
        team.record.losses AS team_losses,
        team.record.ties AS team_ties,
        team.result.outcome AS team_outcome,
        team.result.gamewins AS team_gamewins,
        player.id AS player_id,
        player.role AS player_role
    FROM tourney t
    JOIN unnested_tournaments tr ON tr.tournament_id = t.id  -- Joining on the tournament_id to get the region
    CROSS JOIN UNNEST(stages) AS t (stage)
    CROSS JOIN UNNEST(stage.sections) AS s (section)
    CROSS JOIN UNNEST(section.matches) AS m (match_item)
    CROSS JOIN UNNEST(match_item.teams) AS tm (team)
    CROSS JOIN UNNEST(team.players) AS p (player)
),
teamWins AS (
    SELECT distinct team_id, lol.teams.slug, tourney_matches.region, lol.teams.name, acronym, team_wins, team_losses, team_ties, team_gamewins 
    FROM tourney_matches
    JOIN lol.teams 
    USING(team_id)
),
teamStats AS (
    select region, lol.teams.slug, teamWins.name, teamWins.acronym, team_id, sum(team_wins) as nwin, sum(team_losses) nloss
    from teamWins
    join lol.teams using(team_id)
    group by team_id, lol.teams.slug, region, teamWins.name, teamWins.acronym
)
SELECT * FROM teamStats

"""
    database = "lol"
    s3_output = "s3://query-results-144/a/Dont-bill-me/"
    
    # Execute the query
    query_id = run_query(query, database, s3_output)
    result = get_results(query_id)

    team_dat = []
    headers = []

    # Extract the response form athena/boto3 query
    for i, Rows in enumerate(result['ResultSet']['Rows']):
        if i == 0:
            for El in Rows['Data']:
                val = El['VarCharValue']
                headers.append(val)
            continue
        team_dat.append({f'{header}':Rows['Data'][i]['VarCharValue'] for i,header in enumerate(headers)})
        
    # Store in pandas df  
    team_stats = pd.DataFrame(team_dat)
    
    team_stats["nwin"] = team_stats["nwin"].astype(int)
    team_stats["nloss"] = team_stats["nloss"].astype(int)
    
    team_stats["ntot"] = team_stats["nwin"] + team_stats["nloss"]
    team_stats["winp"] = team_stats["nwin"] / team_stats["ntot"]
    
    international_teams = team_stats[team_stats["region"] == "INTERNATIONAL"]
    
    domestic_counterparts = team_stats[team_stats["slug"].isin(international_teams["slug"]) & 
                                  (team_stats["region"] != "INTERNATIONAL")]

    agg_international_stats = international_teams.merge(domestic_counterparts[["slug", "region"]],
                                                       on="slug", 
                                                       suffixes=("_intl", "_domestic"))
    
    regional_international_winp = agg_international_stats.groupby("region_domestic")["winp"].mean().reset_index()

    regional_international_winp.columns = ["region", "international_winp"]
    
    LatinAmericaRegionalRating = float(regional_international_winp[regional_international_winp["region"] == "LATIN AMERICA"]['international_winp'])

    new_entries = [{"region": "LATIN AMERICA NORTH", "international_winp": LatinAmericaRegionalRating},
                   {"region": "LATIN AMERICA SOUTH", "international_winp": LatinAmericaRegionalRating}]

    regional_international_winp = pd.concat([regional_international_winp, pd.DataFrame(new_entries)], ignore_index=True)
    
    team_stats_with_regional_strength = team_stats.merge(regional_international_winp)

    # get a median strength for the regions that don't have international instances
    median_strength = regional_international_winp["international_winp"]
    
    team_stats_with_regional_strength = team_stats.merge(regional_international_winp, on="region", how="left")

    # for regions without international experience, fill with the median strength
    team_stats_with_regional_strength["international_winp"].fillna(median_strength, inplace=True)
    
    team_stats_with_regional_strength = team_stats_with_regional_strength[team_stats_with_regional_strength["region"] != "INTERNATIONAL"]
    
    team_stats_with_regional_strength["dominance"] = team_stats_with_regional_strength["nwin"] - team_stats_with_regional_strength["nloss"]
    team_stats_with_regional_strength["consistency"] = team_stats_with_regional_strength["winp"]
    team_stats_with_regional_strength["regional_strength"] = team_stats_with_regional_strength["international_winp"]

    
    if event.get('queryStringParameters') and 'dominance' in event['queryStringParameters']:
        dominance = float(event['queryStringParameters']['dominance'])
    else:
        dominance = 0.5
    
    if event.get('queryStringParameters') and 'consistency' in event['queryStringParameters']:
        consistency = float(event['queryStringParameters']['consistency'])
    else:
        consistency = 0.5
        
    if event.get('queryStringParameters') and 'regional_strength' in event['queryStringParameters']:
        regional_strength = float(event['queryStringParameters']['regional_strength'])
    else:
        regional_strength = 1
        
    if event.get('queryStringParameters') and 'streak_bonus' in event['queryStringParameters']:
        streak_bonus = float(event['queryStringParameters']['streak_bonus'])
    else:
        streak_bonus = 0.1
        
    if event.get('queryStringParameters') and 'streak_cutoff' in event['queryStringParameters']:
        streak_cutoff = float(event['queryStringParameters']['streak_cutoff'])
    else:
        streak_cutoff = 0.9
        
    if event.get('queryStringParameters') and 'underdog_bonus' in event['queryStringParameters']:
        underdog_bonus = float(event['queryStringParameters']['underdog_bonus'])
    else:
        underdog_bonus = 0.9
        
    if event.get('queryStringParameters') and 'int_underdog_cutoff' in event['queryStringParameters']:
        int_underdog_cutoff = float(event['queryStringParameters']['int_underdog_cutoff'])
    else:
        int_underdog_cutoff = 0.7
        
    if event.get('queryStringParameters') and 'reg_underdog_cutoff' in event['queryStringParameters']:
        reg_underdog_cutoff = float(event['queryStringParameters']['reg_underdog_cutoff'])
    else:
        reg_underdog_cutoff = 0.4
        
    weights = {
               "dominance": dominance, 
               "consistency": consistency, 
               "regional_strength": regional_strength, 
               "streak_bonus": streak_bonus, 
               "streak_cutoff": streak_cutoff, 
               "underdog_bonus": underdog_bonus,
               "int_underdog_cutoff": int_underdog_cutoff,
               "reg_underdog_cutoff": reg_underdog_cutoff
              }
    
    print(weights)
    
    team_stats_with_regional_strength["base_score"] = (
        weights["dominance"] * team_stats_with_regional_strength["dominance"] +
        weights["consistency"] * team_stats_with_regional_strength["consistency"] +
        weights["regional_strength"] * team_stats_with_regional_strength["regional_strength"]
    )
    
    streak_bonus_mask = team_stats_with_regional_strength["winp"] > weights["streak_cutoff"]
    team_stats_with_regional_strength.loc[streak_bonus_mask, "base_score"] *= 1 + weights["streak_bonus"]
    
    underdog_mask = (team_stats_with_regional_strength["international_winp"] < weights['int_underdog_cutoff']) & (team_stats_with_regional_strength["winp"] > weights['reg_underdog_cutoff'])
    team_stats_with_regional_strength.loc[underdog_mask, "base_score"] *= 1 + weights["underdog_bonus"]
    
    ranked_teams_innovative = team_stats_with_regional_strength.sort_values(by="base_score", ascending=False)

    ranked_teams_innovative["rank"] = range(1, len(ranked_teams_innovative) + 1)

    ranked_teams = ranked_teams_innovative[["rank", "name", "acronym", "team_id", "base_score", "dominance", "consistency", "region", "international_winp"]]
    
    if event.get('queryStringParameters') and 'number_of_teams' in event['queryStringParameters']:
        number_of_teams = int(event['queryStringParameters']['number_of_teams'])
    else:
        number_of_teams = 10

    response_data = [
        {
            "team_id": ranked_teams.iloc[idx]['team_id'],
            "team_code": ranked_teams.iloc[idx]['acronym'],
            "team_name": ranked_teams.iloc[idx]['name'],
            "rank": int(ranked_teams.iloc[idx]['rank']),
            "base_score": float(ranked_teams.iloc[idx]['base_score']),
            "dominance": float(ranked_teams.iloc[idx]['dominance']),
            "consistency": float(ranked_teams.iloc[idx]['consistency']),
            "region": ranked_teams.iloc[idx]['region']
        }
        for idx in range(min(number_of_teams, len(ranked_teams)))
    ]
    
    return {
        'statusCode': 200,
        'body': json.dumps(response_data),
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'
            
        }
    }

In [4]:
request = {
    'queryStringParameters': {
        'number_of_teams': 3
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

response

{'dominance': 0.5, 'consistency': 0.5, 'regional_strength': 1, 'streak_bonus': 0.1, 'streak_cutoff': 0.9, 'underdog_bonus': 0.9, 'int_underdog_cutoff': 0.7, 'reg_underdog_cutoff': 0.4}
Lambda hanlder took  6.41 secods.


{'statusCode': 200,
 'body': '[{"team_id": "102235771678061291", "team_code": "IW", "team_name": "DenizBank \\u0130stanbul Wildcats", "rank": 1, "base_score": 129.75855096009337, "dominance": 135.0, "consistency": 0.7490774907749077, "region": "EMEA"}, {"team_id": "98767991877340524", "team_code": "C9", "team_name": "Cloud9", "rank": 2, "base_score": 123.98655608076564, "dominance": 129.0, "consistency": 0.6996904024767802, "region": "NORTH AMERICA"}, {"team_id": "104367068120825486", "team_code": "PSG", "team_name": "PSG Talon", "rank": 3, "base_score": 118.10613919413919, "dominance": 123.0, "consistency": 0.8153846153846154, "region": "HONG KONG, MACAU, TAIWAN"}]',
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'}}

### Testing custom weights

- dominance: 0.2

- consistency: 0.5
- regional_strength: 0.2
- streak_bonus: 0.2
- streak_cutoff: 0.7
- underdog_bonus: 1.5
- int_underdog_cutoff: 0.4
- reg_underdog_cutoff: 0.6

In [5]:
request = {
    'queryStringParameters': {
        'number_of_teams': 3,
        'dominance': 0.2,
        'consistency': 0.5,
        'regional_strength': 0.2,
        'streak_bonus': 0.2,
        'streak_cutoff': 0.7,
        'underdog_bonus': 0.9,
        'int_underdog_cutoff': 0.4,
        'reg_underdog_cutoff': 0.6
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

json.loads(response['body'])

{'dominance': 0.2, 'consistency': 0.5, 'regional_strength': 0.2, 'streak_bonus': 0.2, 'streak_cutoff': 0.7, 'underdog_bonus': 0.9, 'int_underdog_cutoff': 0.4, 'reg_underdog_cutoff': 0.6}
Lambda hanlder took  6.40 secods.


[{'team_id': '104367068120825486',
  'team_code': 'PSG',
  'team_name': 'PSG Talon',
  'rank': 1,
  'base_score': 57.13310417582417,
  'dominance': 123.0,
  'consistency': 0.8153846153846154,
  'region': 'HONG KONG, MACAU, TAIWAN'},
 {'team_id': '100285330168091787',
  'team_code': 'DFM',
  'team_name': 'DetonatioN FocusMe',
  'rank': 2,
  'base_score': 48.92021835184103,
  'dominance': 105.0,
  'consistency': 0.7966101694915254,
  'region': 'JAPAN'},
 {'team_id': '99566406332987990',
  'team_code': 'CHF',
  'team_name': 'The Chiefs',
  'rank': 3,
  'base_score': 42.92670370370371,
  'dominance': 92.0,
  'consistency': 0.7839506172839507,
  'region': 'OCEANIA'}]

### Aggressive Streak Value

- dominance: 0.6
- consistency: 0.2
- regional_strength: 0.1
- streak_bonus: 0.6
- underdog_bonus: 0.1
- streak_cutoff: 0.8
- underdog_crit_min: 0.2
- underdog_crit_max: 0.5

**Description:**

This distribution heavily prioritizes teams that show strong dominance in their performance. Additionally, teams with a winning percentage greater than 80% (due to the streak_cutoff of 0.8) will get a significant bonus. Underdogs, in this case, are teams with an international win percentage between 20% and 50%. However, the bonus for underdogs is minimal in this distribution.

In [6]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.6,
        'consistency': 0.2,
        'regional_strength': 0.1,
        'streak_bonus': 0.6,
        'streak_cutoff': 0.8,
        'underdog_bonus': 0.1,
        'int_underdog_cutoff': 0.2,
        'reg_underdog_cutoff': 0.5
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.6, 'consistency': 0.2, 'regional_strength': 0.1, 'streak_bonus': 0.6, 'streak_cutoff': 0.8, 'underdog_bonus': 0.1, 'int_underdog_cutoff': 0.2, 'reg_underdog_cutoff': 0.5}
Lambda hanlder took  6.42 secods.
1: PSG Talon
2: Gen.G
3: T1
4: DenizBank İstanbul Wildcats
5: Cloud9
6: DetonatioN FocusMe
7: Zero Tenacity
8: The Chiefs
9: G2 Esports
10: DRX Challengers


### Underdog Favoritism

- dominance: 0.3
- consistency: 0.3
- regional_strength: 0.2
- streak_bonus: 0.1
- underdog_bonus: 0.8
- streak_cutoff: 0.7
- underdog_crit_min: 0.1
- underdog_crit_max: 0.4

**Description:**

This distribution gives a significant advantage to underdog teams with an international win percentage between 10% and 40%. Such teams receive a substantial bonus. Consistency and dominance are equally valued.

In [7]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.3,
        'consistency': 0.3,
        'regional_strength': 0.2,
        'streak_bonus': 0.1,
        'streak_cutoff': 0.7,
        'underdog_bonus': 0.8,
        'int_underdog_cutoff': 0.1,
        'reg_underdog_cutoff': 0.4
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.3, 'consistency': 0.3, 'regional_strength': 0.2, 'streak_bonus': 0.1, 'streak_cutoff': 0.7, 'underdog_bonus': 0.8, 'int_underdog_cutoff': 0.1, 'reg_underdog_cutoff': 0.4}
Lambda hanlder took  6.38 secods.
1: Gen.G
2: T1
3: DenizBank İstanbul Wildcats
4: PSG Talon
5: Cloud9
6: DetonatioN FocusMe
7: G2 Esports
8: Zero Tenacity
9: The Chiefs
10: DRX Challengers


### Balanced Approach

- dominance: 0.4
- consistency: 0.4
- regional_strength: 0.2
- streak_bonus: 0.4
- underdog_bonus: 0.4
- streak_cutoff: 0.7
- underdog_crit_min: 0.3
- underdog_crit_max: 0.6

**Description:**

A balanced distribution where dominance and consistency are equally important. Teams with a winning percentage greater than 70% receive a moderate bonus, and underdogs (with international win percentages between 30% and 60%) are also given moderate consideration.

In [8]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.4,
        'consistency': 0.4,
        'regional_strength': 0.2,
        'streak_bonus': 0.4,
        'streak_cutoff': 0.7,
        'underdog_bonus': 0.4,
        'int_underdog_cutoff': 0.3,
        'reg_underdog_cutoff': 0.6
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.4, 'consistency': 0.4, 'regional_strength': 0.2, 'streak_bonus': 0.4, 'streak_cutoff': 0.7, 'underdog_bonus': 0.4, 'int_underdog_cutoff': 0.3, 'reg_underdog_cutoff': 0.6}
Lambda hanlder took  6.36 secods.
1: PSG Talon
2: Gen.G
3: T1
4: DetonatioN FocusMe
5: DenizBank İstanbul Wildcats
6: The Chiefs
7: G2 Esports
8: Sengoku Gaming
9: Cloud9
10: AGO Rogue


### Emphasis on Regional Strength

- dominance: 0.2
- consistency: 0.2
- regional_strength: 0.6
- streak_bonus: 0.3
- underdog_bonus: 0.2
- streak_cutoff: 0.75
- underdog_crit_min: 0.25
- underdog_crit_max: 0.55

**Description:**

This distribution heavily values the strength of the region from which the team originates. Teams from stronger regions will rank higher. Streaks and underdog performances are considered but are secondary.

In [9]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.2,
        'consistency': 0.2,
        'regional_strength': 0.6,
        'streak_bonus': 0.3,
        'streak_cutoff': 0.75,
        'underdog_bonus': 0.2,
        'int_underdog_cutoff': 0.25,
        'reg_underdog_cutoff': 0.55
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.2, 'consistency': 0.2, 'regional_strength': 0.6, 'streak_bonus': 0.3, 'streak_cutoff': 0.75, 'underdog_bonus': 0.2, 'int_underdog_cutoff': 0.25, 'reg_underdog_cutoff': 0.55}
Lambda hanlder took  6.37 secods.
1: Gen.G
2: PSG Talon
3: T1
4: The Chiefs
5: DetonatioN FocusMe
6: DenizBank İstanbul Wildcats
7: Cloud9
8: Zero Tenacity
9: G2 Esports
10: DRX Challengers


### Underdogs!

Here are a few underdogs. The main idea for this distrbution is to set

- regional_strength: 0.0
- streak_bonus: 0.0
- underdog_bonus: 3.7
- underdog_crit_min: 0.5
- underdog_crit_max: 0.0

Then we look at a few variations on consistency and dominance.

**Description:**

🚀 Introducing the Underdog Rankings! 🚀

The teams that, against all odds, punch above their weight and leave us awestruck? It's time they get the limelight they truly deserve!

With these weights, teams that have a regional international avergae win percentage below 50%, are celebrated with generous bonuses.
🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥

🌟 It's not just about the giants anymore; the stage is set for the underdogs. This is their moment. This is their time to shine! 🌟

In [10]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.5,
        'consistency': 1.2,
        'regional_strength': 0.0,
        'streak_bonus': 0.0,
        'underdog_bonus': 3.7,
        'int_underdog_cutoff': 0.5,
        'reg_underdog_cutoff': 0.0
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.5, 'consistency': 1.2, 'regional_strength': 0.0, 'streak_bonus': 0.0, 'streak_cutoff': 0.9, 'underdog_bonus': 3.7, 'int_underdog_cutoff': 0.5, 'reg_underdog_cutoff': 0.0}
Lambda hanlder took  6.42 secods.
1: DenizBank İstanbul Wildcats
2: Cloud9
3: PSG Talon
4: DetonatioN FocusMe
5: Zero Tenacity
6: G2 Esports
7: Sengoku Gaming
8: The Chiefs
9: AGO Rogue
10: LDLC OL


In [11]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.0,
        'consistency': 0.1,
        'regional_strength': 0.0,
        'streak_bonus': 0.0,
        'underdog_bonus': 3.5,
        'int_underdog_cutoff': 0.5,
        'reg_underdog_cutoff': 0.0
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.0, 'consistency': 0.1, 'regional_strength': 0.0, 'streak_bonus': 0.0, 'streak_cutoff': 0.9, 'underdog_bonus': 3.5, 'int_underdog_cutoff': 0.5, 'reg_underdog_cutoff': 0.0}
Lambda hanlder took  6.47 secods.
1: UST Esports
2: Primate
3: GAM Esports
4: PSG Talon
5: DetonatioN FocusMe
6: The Chiefs
7: Team Bliss
8: Cloud9 Challengers
9: Mirage Alliance
10: Bandits Gaming


In [12]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.0,
        'consistency': 0.0,
        'regional_strength': 0.0,
        'streak_bonus': 0.0,
        'underdog_bonus': 3.5,
        'int_underdog_cutoff': 0.5,
        'reg_underdog_cutoff': 0.0
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.0, 'consistency': 0.0, 'regional_strength': 0.0, 'streak_bonus': 0.0, 'streak_cutoff': 0.9, 'underdog_bonus': 3.5, 'int_underdog_cutoff': 0.5, 'reg_underdog_cutoff': 0.0}
Lambda hanlder took  6.34 secods.
1: PIGSPORTS
2: Gentle Hearts Gaming
3: Maturalni Forsaken
4: FLY FAM
5: The Last Dance
6: paiN Academy
7: Los Grandes Academy
8: FlyQuest Challengers
9: 100 Thieves Challengers
10: Cincinnati Fear


In [13]:
request = {
    'queryStringParameters': {
        'number_of_teams': 10,
        'dominance': 0.1,
        'consistency': 0.0,
        'regional_strength': 0.0,
        'streak_bonus': 0.0,
        'underdog_bonus': 3.5,
        'int_underdog_cutoff': 0.5,
        'reg_underdog_cutoff': 0.0
    }
}


start_time = time.time()
response = lambda_handler(request, 0)
end_time = time.time()

print(f"Lambda hanlder took {end_time - start_time: 0.2f} secods.")

for t in json.loads(response['body']):
    print(f'{t["rank"]}: {t["team_name"]}')

{'dominance': 0.1, 'consistency': 0.0, 'regional_strength': 0.0, 'streak_bonus': 0.0, 'streak_cutoff': 0.9, 'underdog_bonus': 3.5, 'int_underdog_cutoff': 0.5, 'reg_underdog_cutoff': 0.0}
Lambda hanlder took  6.41 secods.
1: DenizBank İstanbul Wildcats
2: Cloud9
3: PSG Talon
4: DetonatioN FocusMe
5: Zero Tenacity
6: G2 Esports
7: Sengoku Gaming
8: The Chiefs
9: LDLC OL
10: AGO Rogue
